# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [3]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

In [4]:
import pandas as pd
import nltk
import numpy as np
import scipy
from nltk.metrics.distance import edit_distance
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [5]:
# Загрузка данных
data = pd.read_csv("preprocessed_descriptions.csv")
descriptions = data["preprocessed_descriptions"].dropna().tolist()
# Получение уникальных слов
unique_words = set()

for description in descriptions:
    words = nltk.word_tokenize(description)
    unique_words.update(words)
    
words = list(unique_words)
print(words)

['consulting', 'ceecee526', 'transcribe', 'divinefor', 'missy', 'tastyserve', 'skopelos', 'linguine', 'peppery', 'todays', '66870', 'morewithless', 'baylesss', 'nothing', 'lasagnamaking', 'kidsthey', '06g', 'rounded', 'kpds', 'mill', 'surfaces', 'ithehe', 'shortenings', 'thickcut', '70', 'bust', 'authentically', 'carried', 'reiner', 'starledger', 'joan', 'floddies', 'burger', 'israelfoods', 'calzone', 'worlds', 'booklet', 'mawmaws', '22213', 'palomilla', 'hasa', 'sinewy', 'shrimpand', 'stellar', 'alternately', 'ration', 'dividing', 'fixings', 'triplelayered', 'temperance', 'confetti', 'free', 'saladsthey', 'pike', 'caterpillar', 'breadman', 'bursting', 'amts', 'partook', 'tears', 'quaint', 'beega', 'substitution', 'kall', 'normally', 'reconstitute', 'youwe', 'dessertlovers', 'enticed', 'webiste', 'nelson', 'rownothing', 'abbundance', 'citing', 'schoolwork', 'waerebeeks', 'romeos', 'zmail', 'nutritioushigh', 'barneys', 'soakings', 'salvador', 'porkveal', 'andes', 'morish', 'pails', '2ye

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [6]:
sample = np.random.choice(np.array(words, str), 10, replace=False).reshape((5,2))

for i in sample:
    print(i, edit_distance(i[0], i[1]))

['hotthe' 'crepelike'] 8
['disasters' '111984'] 9
['exchange2' 'lan'] 7
['nobu' 'craig'] 5
['timer' 'okeefe'] 5


In [7]:
print(words)

['consulting', 'ceecee526', 'transcribe', 'divinefor', 'missy', 'tastyserve', 'skopelos', 'linguine', 'peppery', 'todays', '66870', 'morewithless', 'baylesss', 'nothing', 'lasagnamaking', 'kidsthey', '06g', 'rounded', 'kpds', 'mill', 'surfaces', 'ithehe', 'shortenings', 'thickcut', '70', 'bust', 'authentically', 'carried', 'reiner', 'starledger', 'joan', 'floddies', 'burger', 'israelfoods', 'calzone', 'worlds', 'booklet', 'mawmaws', '22213', 'palomilla', 'hasa', 'sinewy', 'shrimpand', 'stellar', 'alternately', 'ration', 'dividing', 'fixings', 'triplelayered', 'temperance', 'confetti', 'free', 'saladsthey', 'pike', 'caterpillar', 'breadman', 'bursting', 'amts', 'partook', 'tears', 'quaint', 'beega', 'substitution', 'kall', 'normally', 'reconstitute', 'youwe', 'dessertlovers', 'enticed', 'webiste', 'nelson', 'rownothing', 'abbundance', 'citing', 'schoolwork', 'waerebeeks', 'romeos', 'zmail', 'nutritioushigh', 'barneys', 'soakings', 'salvador', 'porkveal', 'andes', 'morish', 'pails', '2ye

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [8]:
def find_alike(words, word,k):
    distances = dict()
    for w in words:
        try:
            distances[edit_distance(w, word , substitution_cost=2, transpositions=True)].append(w)
        except:
            distances[edit_distance(w, word , substitution_cost=2, transpositions=True)] = [w]
            
    return distances[k]

find_alike(words,'but', 1)

['bust', 'butt', 'ut', 'burt', 'buti', 'bunt']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [9]:
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

words = np.array(words)
stemmed_words = np.array([stemmer.stem(_) for _ in words])
normalized_words = np.array([lemmatizer.lemmatize(_) for _ in words])

two_one = (pd.DataFrame(np.stack((words,stemmed_words,normalized_words))).T)
two_one.columns = ([['word','stemmed_word','normalized_word']])
two_one.set_index('word', inplace=True)

two_one

,stemmed_word,normalized_word
word,,
"(consulting,)",consult,consulting
"(ceecee526,)",ceecee526,ceecee526
"(transcribe,)",transcrib,transcribe
"(divinefor,)",divinefor,divinefor
"(missy,)",missi,missy
...,...,...
"(cookbooklet,)",cookbooklet,cookbooklet
"(sillyyaks,)",sillyyak,sillyyaks
"(face,)",face,face


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [10]:
en_stop_words = stopwords.words('english')

data = pd.read_csv("preprocessed_descriptions.csv")
descriptions = data["preprocessed_descriptions"].dropna().tolist()

words_dict = dict()
st_words_dict = dict()

for description in descriptions:
    words = nltk.word_tokenize(description)
    for word in words:
        try:
            words_dict[word]+=1
        except:
            words_dict[word]=1
        if word not in en_stop_words:
            try:
                st_words_dict[word]+=1
            except:
                st_words_dict[word]=1

sorted_dict = {x: words_dict[x] for x in sorted(words_dict.keys(), key=lambda ele: words_dict[ele], reverse=True)}
sorted_st_dict = {x: st_words_dict[x] for x in sorted(st_words_dict.keys(), key=lambda ele: st_words_dict[ele], reverse=True)}

print(list(sorted_dict.keys())[0:10])
print(list(sorted_st_dict.keys())[0:10])

['the', 'a', 'and', 'this', 'i', 'to', 'is', 'it', 'of', 'for']
['recipe', 'make', 'time', 'use', 'great', 'like', 'easy', 'one', 'made', 'good']


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [43]:
data = pd.read_csv("preprocessed_descriptions.csv")
ds = data.sample(5)

sample = ds["preprocessed_descriptions"].to_numpy()

tv = TfidfVectorizer()
sample_tv = tv.fit_transform(sample)
sample_array = sample_tv.toarray()

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [47]:
mas = pd.DataFrame(np.array([[scipy.spatial.distance.cosine(sample_array[i], sample_array[j]) for j in range(len(sample_array))] for i in range(len(sample_array))]), columns=ds["name"].to_numpy(),index=ds["name"].to_numpy())
mas

,curried cheese bacon chicken,chili crock pot,grilled halloumi and steak kabobs,cream of corn and sweet red pepper,jayme s sloppy joes
curried cheese bacon chicken,0.000000,0.784308,0.947529,0.964043,0.931023
chili crock pot,0.784308,0.000000,0.934068,0.841447,0.815468
grilled halloumi and steak kabobs,0.947529,0.934068,0.000000,0.947305,0.903331
cream of corn and sweet red pepper,0.964043,0.841447,0.947305,0.000000,0.946325
jayme s sloppy joes,0.931023,0.815468,0.903331,0.946325,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

Наиболее похожими являются рецепты из максимально схожих слов.